# Trabajando con datos raster

Este ejercitario utiliza las tareas comunes de procesar datos espaciales raster que incluyen:

- Abrir datos geográficos desde imagenes
- Visualizar información basica
- Realizar analisis espacial 

Para responder cada pregunta trate crear celdas independientes.

Es necesario importar los siguientes paquetes iniciales para trabajar.

In [ ]:
import rasterio # para abrir datos raster
import rasterio.plot as rplt # Graficas con Rasterio
import rasterio.mask as rmask # Mascaras con Rasterio
import matplotlib.pyplot as plt # visualizar informacion
import geopandas # para leer datos espaciales en tablas
import pandas # para leer datos en tablas
import numpy as np

# Para desplegar paginas web
from IPython.display import IFrame
%matplotlib inline

## Indicaciones
En analisis de raster usualmente el analista ya posee datos preparados para trabajar con estos. Puede que la información este disponible en formatos conocidos (Ej. shapefiles, raster tiff, geopackage .gpkg o geojson). 

A continuación, utilizaremos unos datos de ejemplo de [Global Forest Change](http://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.6.html), con información de detección de deforestación por año en la grilla de sudamérica 20S 60W. Esta información sirve como ejemplo de datos que son distribuidos en un formato espacial explicito. También se provee los departamentos de paraguay en poligonos.

Nota: Si no encuentra el archivo `Hansen_GFC-2018-v1.6_lossyear_20S_060W.tif` asegurese de descargar los archivos `paraguay_dpto_WGS84.zip` y https://storage.googleapis.com/earthenginepartners-hansen/GFC-2018-v1.6/Hansen_GFC-2018-v1.6_lossyear_20S_060W.tif necesarios, y extrer los mismos en la ubicación `datos/`.

## Preparar Datos

### 1. Abrir datos raster TIF

Utilice el archivo `datos/Hansen_GFC-2018-v1.6_lossyear_20S_060W.tif` y abralo con `rasterio.open` para crear una dataset de datos en Numpy.

Puede verificar la ayuda ejecutando con `rasterio.open?`, una de las opciones disponibles es `nodata=` que permite establecer el valor de "SIN DATOS". 

Segun la informacion de Global Forest Watch. El archivo posee valores del 1 al 17 en los pixeles representando el cambio para los años 2001 - 2017, y 0 seria "Sin Cambios".

Almacena la informacion en la variable `defor_sur` y verifica la informacion con `.profile` para responder las siguientes preguntas:

- ¿Cuantas filas y columnas posee el dataset?
- ¿Cual es el sistema de referencia de los dataset?
- ¿Cuantas bandas posee el dataset?
- ¿Que tipo de datos utiliza cada celda?
- ¿Que resolucion espacial posee cada celda? Exprese su respuesta en metros dependiendo de la unidad de medida del CRS


Utiliza las celdas de abajo para justificar la respuesta con el código necesario.

### 2. Abrir un GeoDataFrame con datos de departamentos
Con el archivo `paraguay_dpto_WGS84.shp` se va a utilizar las geometrias para un analisis mas minucioso. En la variable `py_dpto` se puede abrir la informacion con `geopandas.GeoDataFrame`, utilice el metodo `geopandas.read_file`: Ej.

```python
py_dpto = geopandas.read_file("datos/paraguay_dpto_WGS84.shp")
```

Para completar este paso se debe:

- Abrir en un `GeoDataFrame` el archivo de los departamentos del Paraguay. Verificar la información con el método `.head()`.
- Que sistema de referencia posee el dataframe `py_dpto`?.

_Nota_: 
Asegurese que coincidan los sistemas de referencia CRS tanto del archivo TIF como del SHAPE.

### 3. Analizar información
Ahora que tiene los departamentos y la deforestacion se puede resumir los datos en base al area de estudio.

#### Enfoque en area de interes
Se enfoca el estudio en el area de Paraguay, acotando los datos raster a solo esta extension por medio de una "Ventana". Vea la informacion de `defo_usr.window` y `defo_sur.read?` para obtener mas detalles. Ej.

```python
ext_py = py_dpto.total_bounds
window_py = defor_sur.window(ext_py[0], ext_py[1], ext_py[2], ext_py[3])
defor_py = defor_sur.read(1, window=window_py, masked=True)
```

Se utiliza el parametro `masked=True` para que los datos puedan ser filtrados con mascara para excluir celdas no deseadas. En esta caso se limpian las celdas para solo incluir celdas con valores del 1 al 17.

```python
defor_py.mask = defor_py < 1
```

Se debe analizar la informacion con los metodos `.min()`, `max()` y `.shape` y responder cuanto sigue:

- Cual es el valor minimo de cada celda y a que correponde?
- Cual es el valor maximo de cada celda y a que correponde?
- Cuantas filas y columnas posee el raster, y cuantas celdas?


#### Histograma
Crear un histograma luego del contexto espacial del paraguay y el filtro de sin cambios. 
Para esto primero calcule la tabla de frecuencias con el siguiente codigo:

```python
freq = []
for i in range(defor_py.min(), defor_py.max()+1):
    freq.append((2000 + i, defor_py[defor_py == i].count()))
resumen = pandas.DataFrame(data=freq, columns=("year", "freq"))
```

Se crea una variable `resumen` con la cantidad de celdas por año de donde se puede estimar las hectareas por año deforestadas.

Para visualizar realice el siguiente histograma:

```python
resumen.plot(kind="bar", y="freq", x="year",  color="red", title="Deforestacion por año")
```

Se requiere lo siguiente (crear una celda independiente para las graficas y resultados):

- Analice la informacion del histograma y responda, cual es el año con mayor deforestación?
- Cree una columna en la tabla `resumen` llamada "hectarea" y calcule a partir de la columna "freq" las hectareas deforestadas.
- Cuantas hectareas fueron desmontadas en el año de mayor deforestación?
- Visualice la tabla `resumen` en una celda independiente.


_Nota_: 
Puede verificar el valor maximo con el método describe() o por medio de un filtro con el método idxmax().

Para el calculo de hectareas tenga en cuenta que `1 m = 0.0001 ha`, utilice el valor de la resolucion espacial de las celdas en metros `m` encontrado en la primera parte.

## Resumir Informacion

El ultimo paso seria resumir de manera grafica y tabular la deforestacion en hectareas de algunos departamentos por año en Paraguay. Para esto necesitara utilizar el shapefile de los departamentos `py_dpto`. 

La informacion geografica puede ser verificada con la propiedad .crs para ver que coincida con el TIF.

### 1. Extraer una region del raster
Para realizar una extraccion y calculo por valores dentro de una area se utiliza la informacion de `py_dpto` con los departamentos. Verifique la informacion con `.head()`, la columna "dpto_desc" contiene los nombres de los departamentos y procederemos a filtrar por "CANINDEYU".

Se utiliza el paquete `rasterio.mask` con el nombre `rmask` para la extraccion por informacion geografica y `numpy.ma` para filtrar en conjunto los valores "Sin Cambios".

```python
area_interes = py_dpto[py_dpto.dpto_desc == "CANINDEYU"].geometry
defor_cyu, trans_cyu = rmask.mask(defor_sur, shapes=area_interes, crop=True, all_touched=True, filled=False, indexes=1)
defor_cyu = np.ma.masked_less(defor_cyu, 1)
```

Se requiere contestar las siguientes preguntas:

- Cual es el valor minimo de cada celda?
- Cual es el valor maximo de cada celda?
- Cuantas filas y columnas posee el raster, y cuantas celdas?

### 2. Resumir Informacion
Crear un histograma luego en el area espacial de CANINDEYU y el filtro de sin cambios. 
Para esto primero calcule la tabla de frecuencias con el siguiente codigo:

```python
freq = []
for i in range(defor_cyu.min(), defor_cyu.max()+1):
    freq.append((2000 + i, defor_cyu[defor_cyu == i].count()))
resumen_cyu = pandas.DataFrame(data=freq, columns=("year", "freq"))
```

Se crea una variable `resumen_cyu` con la cantidad de celdas por año de donde se puede estimar las hectareas por año deforestadas.

Para resumir la informacion realice el siguiente histograma:

```python
resumen_cyu.plot(kind="bar", y="freq", x="year", color="red", title="Deforestacion por año en CANINDEYU")
```

Se requiere lo siguiente (crear una celda independiente para las graficas y resultados):

- Analice la informacion del histograma y responda, cual es el año con mayor deforestación?
- Cree una columna en la tabla `resumen_cyu` llamada "hectarea" y calcule a partir de la columna "freq" las hectareas deforestadas.
- Cuantas hectareas fueron desmontadas en el año de mayor deforestación en CANINDEYU?
- Visualice la tabla `resumen_cyu` en una celda independiente.


_Nota_: 
Puede verificar el valor maximo con el método describe() o por medio de un filtro con el método idxmax().

Para el calculo de hectareas tenga en cuenta que `1 m = 0.0001 ha`, utilice el valor de la resolucion espacial de las celdas en metros `m` encontrado en la primera parte.

### 3. Visualizar Cambios
Crear un histograma en el area espacial de CANINDEYU para identificar las diferentes areas y periodos de cambio.. 
Grafique el mapa en base al siguiente codigo:

```python
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ext = area_interes.total_bounds[[0,2,1,3]]
im = ax.imshow(defor_cyu, cmap='Spectral', extent=ext)
rangos = range(1,19)
plt.colorbar(im, ax=ax, values=rangos, ticks=rangos, orientation='horizontal')
area_interes.boundary.plot(ax=ax, color='grey', linewidth=4)
```

Se requiere lo siguiente (crear una celda independiente para las graficas y resultados):

- Analice la informacion del mapa y responda, cual es el area con mayor cambio a simple vista? Puede usar la referencia en rango de latitudes o longitudes o con el lenguaje Nor-Este, Norte, Sur, etc.
- Que tipo de cambios ocurrieron en el año de mayor deforestacion en cuanto a la distribucion de cambios, Ej. pequeños desmontes distribuidos o grandes areas afectadas.


## Analizar regiones especificas

El ultimo paso seria resumir en un histograma, mapa y tabular la deforestacion en hectareas en otros departamentos por año en Paraguay. Para esto necesitara utilizar el shapefile de los departamentos `py_dpto`. 

### 1. Estudiar el cambio en SAN PEDRO
Analizar el histograma y visualizar los cambios en "SAN PEDRO". Copie las celdas y codigos de los ejercicios anteriores pero en nuevas variables `defor_spo` y `resumen_spo`, responder a las siguientes preguntas: 

- Analice la informacion del histograma, cual es el año con mayor deforestación?
- Analice la informacion del mapa, cual es el area con mayor cambio a simple vista? Puede usar la referencia en rango de latitudes o longitudes o con el lenguaje Nor-Este, Norte, Sur, etc.
- Cree una columna en la tabla `resumen_spo` llamada "hectarea" y calcule a partir de la columna "freq" las hectareas deforestadas.
- Cuantas hectareas fueron desmontadas en el año de mayor deforestación en SAN PEDRO?
- Visualice la tabla `resumen_spo` en una celda independiente.

### 2. Estudiar el cambio en CAAGUAZU 
Analizar el histograma y visualizar los cambios en "CAAGUAZU". Copie las celdas y codigos de los ejercicios anteriores pero en nuevas variables `defor_czu` y `resumen_czu`, responder a las siguientes preguntas: 

- Analice la informacion del histograma, cual es el año con mayor deforestación?
- Analice la informacion del mapa, cual es el area con mayor cambio a simple vista? Puede usar la referencia en rango de latitudes o longitudes o con el lenguaje Nor-Este, Norte, Sur, etc.
- Cree una columna en la tabla `resumen_spo` llamada "hectarea" y calcule a partir de la columna "freq" las hectareas deforestadas.
- Cuantas hectareas fueron desmontadas en el año de mayor deforestación en SAN PEDRO?
- Visualice la tabla `resumen_spo` en una celda independiente.